In [245]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/language-identification/sample_submission.csv
/kaggle/input/language-identification/train_set.csv
/kaggle/input/language-identification/test_set.csv


In [246]:
#!pip install imblearn
#!pip install smote_variants

In [247]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import f1_score, classification_report, precision_score, recall_score, log_loss, make_scorer,confusion_matrix
from sklearn.pipeline import Pipeline
from imblearn.metrics import classification_report_imbalanced

from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold

#import smote_variants as sv

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [248]:
train = pd.read_csv('../input/language-identification/train_set.csv')
test = pd.read_csv('../input/language-identification/test_set.csv')
sample = pd.read_csv('../input/language-identification/sample_submission.csv')

In [249]:
train.head()


,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [250]:
test.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


In [251]:
sample.head()

,index,lang_id
0,1,tsn
1,2,nbl


In [252]:
train_df = train.copy()
train_df["text"].str.lower()

0        umgaqo-siseko wenza amalungiselelo kumaziko ax...
1        i-dha iya kuba nobulumko bokubeka umsebenzi na...
2        the province of kwazulu-natal department of tr...
3        o netefatša gore o ba file dilo ka moka tše le...
4        khomishini ya ndinganyiso ya mbeu yo ewa maana...
                               ...                        
32995    popo ya dipolateforomo tse ke go tlisa boetele...
32996    modise mosadi na o ntse o sa utlwe hore thaban...
32997    closing date for the submission of completed t...
32998    nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999    mafapha a mang le ona a lokela ho etsa ditlale...
Name: text, Length: 33000, dtype: object

In [253]:
test.info

<bound method DataFrame.info of       index                                               text
0         1  Mmasepala, fa maemo a a kgethegileng a letlele...
1         2  Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2         3         Tshivhumbeo tshi fana na ngano dza vhathu.
3         4  Kube inja nelikati betingevakala kutsi titsini...
4         5                      Winste op buitelandse valuta.
...     ...                                                ...
5677   5678                   You mark your ballot in private.
5678   5679  Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679   5680  E Ka kopo etsa kgetho ya hao ka hloko, hobane ...
5680   5681  TB ke bokudi ba PMB, mme Morero o tla lefella ...
5681   5682              Vakatjhela iwebhusayidi yethu ku-www.

[5682 rows x 2 columns]>

In [254]:
print(train_df.shape)
print(test.shape)

display(train_df.head())
display(test.head())

percent_duplicates = round((1-(train_df['text'].nunique()/len(train['text'])))*100,2)
print('Duplicated text in train_df data:')
print(percent_duplicates,'%')

(33000, 2)
(5682, 2)


,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


Duplicated text in train_df data:
9.25 %


In [255]:
train_df = train_df.drop_duplicates(keep='first')

In [256]:
from nltk.corpus import stopwords
nltk.download('stopwords')
#remove stopwords 
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
train_df["text_stop"] = train_df["text"].apply(lambda text: remove_stopwords(text))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [257]:
from collections import Counter
cnt = Counter()
for text in train_df["text_stop"].values:
    for word in text.split():
        cnt[word] +=1


In [258]:
def data_cleaning(y):
    #removing panctuation
    y = re.sub(r"[,.;':'@#?!/$]+\*",' ',y)
    #removing numbers
    y= re.sub(r'\d+','',y)
    #removing extra whitespaces
    y = re.sub(r'\s\s+',' ',y)
    #removing space
    y = y.lstrip(' ')
    return y

In [259]:
train_df['clean_text'] = train_df['text_stop'].apply(data_cleaning)

In [260]:
train_df.shape

(29948, 4)

In [261]:
X = train_df['clean_text']
y=train_df['lang_id']


In [262]:
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=.15,shuffle=True, stratify=y, random_state=11)

In [263]:
param_grid = {'alpha':[0.01, 0.1, 1]}
NB = Pipeline([('NB_tfidf', TfidfVectorizer(ngram_range=(1,3), min_df=2)),
               ('NB_clf',GridSearchCV(MultinomialNB(),
                                   param_grid=param_grid,
                                   cv=11,
                                   n_jobs=-1) )])

NB.fit(X_train, y_train)
y_pred_NB = NB.predict(X_val)


print(classification_report_imbalanced(y_val, y_pred_NB))
NB_metrics = classification_report(y_val, y_pred_NB, output_dict=True)

                   pre       rec       spe        f1       geo       iba       sup

        afr       1.00      1.00      1.00      1.00      1.00      1.00       396
        eng       1.00      1.00      1.00      1.00      1.00      1.00       450
        nbl       1.00      1.00      1.00      1.00      1.00      1.00       354
        nso       1.00      1.00      1.00      1.00      1.00      1.00       431
        sot       1.00      1.00      1.00      1.00      1.00      1.00       425
        ssw       0.99      1.00      1.00      1.00      1.00      1.00       364
        tsn       1.00      1.00      1.00      1.00      1.00      1.00       430
        tso       1.00      1.00      1.00      1.00      1.00      1.00       414
        ven       1.00      1.00      1.00      1.00      1.00      1.00       391
        xho       1.00      1.00      1.00      1.00      1.00      1.00       399
        zul       1.00      1.00      1.00      1.00      1.00      0.99       439

av

In [264]:
#LSVC = Pipeline([('LSVC_tfidf', TfidfVectorizer(min_df=2, smooth_idf=False, ngram_range=(1, 3))),
                 #('LSVC_clf', LinearSVC(random_state=42))])

#LSVC.fit(X_train, y_train)
#y_pred_LSVC = LSVC.predict(X_val)

#print(classification_report_imbalanced(y_val, y_pred_LSVC))
#LSVC_metrics = classification_report_imbalanced(y_val, y_pred_LSVC)

In [265]:
test_df = test.copy()
test_df

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.
...,...,...
5677,5678,You mark your ballot in private.
5678,5679,Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679,5680,"E Ka kopo etsa kgetho ya hao ka hloko, hobane ..."
5680,5681,"TB ke bokudi ba PMB, mme Morero o tla lefella ..."


In [266]:
testX = test_df.text

In [267]:
y_pred = NB.predict(testX)

In [268]:
test_df['lang_id'] = y_pred

In [269]:
test_df.head()

,index,text,lang_id
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele...",tsn
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...,nbl
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.,ven
3,4,Kube inja nelikati betingevakala kutsi titsini...,ssw
4,5,Winste op buitelandse valuta.,afr


In [270]:
test_df[['index','lang_id']].to_csv('submission',index = False)